In [12]:
import cv2
from keras.models import load_model
import numpy as np
from pygame import mixer

In [13]:
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')

model_output = load_model(r'D:\Paul\CHS_Proiect\Models\model.h5')

In [14]:
dizziness_counter = 0

mixer.init()
alarm_sound = mixer.Sound(r'D:\Paul\CHS_Proiect\Proiect_CHS\DizzinessDetection\alarm.wav')
camera = cv2.VideoCapture(0)

while True:
    ret, frame = camera.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    height, width = frame.shape[0:2]

    faces = face_cascade.detectMultiScale(gray, scaleFactor = 1.2, minNeighbors = 3)
    eyes = eye_cascade.detectMultiScale(gray, scaleFactor = 1.1, minNeighbors = 1)

    cv2.rectangle(frame, (0, height-50), (400, height), (0,0,0), thickness=cv2.FILLED)

    #draw face rectangle
    for (x,y,w,h) in faces:
        cv2.rectangle(frame, pt1=(x,y), pt2=(x+w, y+h), color=(255, 0, 0), thickness=2)

    #draw eyes rectangle
    for (ex,ey,ew,eh) in eyes:
        # cv2.rectangle(frame, pt1=(ex,ey), pt2=(ex+ew, ey+eh), color=(255, 0, 0), thickness=2)

        #preprocessing step
        eye = frame[ey:ey+eh, ex:ex+ew]
        eye = cv2.resize(eye, (80,80))
        eye = eye/255
        eye = eye.reshape(80, 80, 3)
        eye = np.expand_dims(eye, axis=0)

        #model prediction
        prediction = model_output.predict(eye)

        #closed eyes
        if prediction[0][0] > 0.30:
            cv2.putText(frame, 'CLOSED eyes!', (10, height-20), fontFace = cv2.FONT_HERSHEY_SIMPLEX, fontScale=1, color = (255,255,255), thickness = 2, lineType=cv2.LINE_AA)
            cv2.putText(frame, 'Score: ' + str(dizziness_counter), (250, height-20), fontFace = cv2.FONT_HERSHEY_SIMPLEX, fontScale=1, color = (255,255,255), thickness = 2, lineType=cv2.LINE_AA)

            dizziness_counter = dizziness_counter + 1
            if(dizziness_counter > 15):
                try:
                    alarm_sound.play()
                except:
                    pass
        #opened eyes
        elif prediction[0][1] > 0.90:
            cv2.putText(frame, 'Opened eyes.', (10, height-20), fontFace = cv2.FONT_HERSHEY_SIMPLEX, fontScale=1, color = (255,255,255), thickness = 2, lineType=cv2.LINE_AA)
            cv2.putText(frame, 'Score: ' + str(dizziness_counter), (250, height-20), fontFace = cv2.FONT_HERSHEY_SIMPLEX, fontScale=1, color = (255,255,255), thickness = 2, lineType=cv2.LINE_AA)

            dizziness_counter = dizziness_counter - 1
            if(dizziness_counter < 0):
                dizziness_counter = 0

    cv2.imshow('frame', frame)
    if cv2.waitKey(33) & 0xFF==ord('s'):
        break

camera.release()
cv2.destroyAllWindows()

1/1 [==============================] - 0s 50ms/step
